In [39]:
import numpy as np
from patchify import patchify
import tifffile as tiff
import os
from PIL import Image, ImageDraw
from math import floor
import random
import shutil
import os
from torchvision import transforms
from utils import make_patches, make_split
from sklearn.model_selection import train_test_split

### Split train data into train-val

In [ ]:
make_split("crow_trap", dataset_dir="Datasets/Crow_detection/train", split_p=0.2)
make_split("background", dataset_dir="Datasets/Crow_detection/train", split_p=0.2)

### Data augmentation

The train split can be used to generate augmented data and reduce imbalance in the dataset.

- **crow_trap generation**: traps are manually masked out and then placed into background images
- **general augmentation**: background/crow_trap images can be transformed and distorted 

##### Crow trap generation

YOLO annotation format (https://medium.com/analytics-vidhya/getting-data-annotation-format-right-for-object-detection-tasks-f41b07eebbf5)
<class> <x> <y> <width> <height> 
ex: 0 0.25 0.44 0.5 0.8
class is the object class, (x,y) are centre coordinates of the bounding box. width, height represent width and height of the bounding box

In [28]:
from math import ceil

In [54]:
def annotate_img(img_filename, center=None, trap_size=None, img_size=(224,224), class_name="0"):
    name = img_filename.split(".")[0]
    f= open(name+".txt","w+")

    if center and trap_size:
        dw = 1. / img_size[0]
        dh = 1. / img_size[1]
        x = center[0] * dw
        y = center[1] * dh
        w = trap_size[0] * dw
        h = trap_size[1] * dh

        annotation = " ".join([class_name, str(x), str(y), str(h), str(w)])
        f.write(annotation)

    f.close()

annotate_img("Datasets/Crow_detection/generated_traps/test_ann.jpg", (81, 118), (17, 17))

In [72]:
def generate_detection_trap_samples(back_dir, trap_dir, save_img_dir, save_ann_dir, save_gt_dir, split_p = 1):

    if not os.path.exists(save_img_dir):
        os.makedirs(save_img_dir)

    if not os.path.exists(save_ann_dir):
        os.makedirs(save_ann_dir)

    if not os.path.exists(save_gt_dir):
        os.makedirs(save_gt_dir)


    transform_p = 0.5

    combined_transforms = transforms.Compose(
        [
            transforms.ColorJitter(
                brightness=0.5, contrast=0.15, saturation=0.3, hue=0.3
            ),
        ]
    )

    back_transforms = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(transform_p),
            transforms.RandomVerticalFlip(transform_p),
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(transform_p),
            transforms.RandomVerticalFlip(transform_p),
        ]
    )

    samples_back = os.listdir(back_dir)
    if split_p<1:
        _, samples_back = train_test_split(os.listdir(back_dir), test_size=split_p, random_state=42)
    print("N samples: ",len(samples_back))

    samples_trap = os.listdir(trap_dir)

    for sample in samples_back:
        # Load and transform background and trap images
        back = Image.open(os.path.join(back_dir, sample))
        back = back_transforms(back)

        sample_trap = random.choice(samples_trap)
        trap = Image.open(os.path.join(trap_dir, sample_trap))

        angle = random.randint(0,360)
        trap.rotate(angle, resample=Image.BICUBIC, expand=True)
        trap = test_transforms(trap)

        # Calculate trap coordinates (top,left) and center
        w, h = trap.size
        left = random.randint(0, back.size[0]-w)
        top = random.randint(0, back.size[1]-h)
        center = (left+int(w/2), top+int(h/2))

        # Combine and save background + trap
        back.paste(trap, (left, top), trap)
        back = combined_transforms(back)

        new_trap_name = sample.split(".")[0] + "_" + sample_trap
        print(new_trap_name)
        annotate_img(os.path.join(save_ann_dir,new_trap_name), center, (w, h), back.size, "0")
        back.save(os.path.join(save_img_dir,new_trap_name))

        # Save GT BB
        img_draw = ImageDraw.Draw(back)
        img_draw.rectangle(((left, top),(left+w, top+h)), outline='Red')
        back.save(os.path.join(save_gt_dir,new_trap_name.split(".")[0]+"_GT.jpeg"))

In [74]:
trap_dir = "Datasets/backup/Only_traps"
back_dir = "Datasets/Crow_detection/train/background"
save_img_dir = "Datasets/Crow_detection_train/images/train"
save_ann_dir = "Datasets/Crow_detection_train/labels/train"
save_gt_dir = "Datasets/Crow_detection/train/GT_BB"

generate_detection_trap_samples(back_dir, trap_dir, save_img_dir, save_ann_dir, save_gt_dir)

N samples:  307
101343_3_1_11.png
100459_3_0_17.png
103638_1_1_3.png
101517_0_2_11.png
101339_2_0_5.png
101546_1_0_4.png
100615_3_0_11.png
104008_3_0_2.png
100980_1_0_17.png
101123_2_2_5.png
103519_0_2_9.png
100615_0_1_15.png
101679_0_1_5.png
101520_0_1_16.png
101343_0_2_18.png
101569_2_2_9.png
104269_1_0_9.png
101442_3_0_13.png
101339_2_2_2.png
101125_0_1_9.png
103303_1_0_13.png
102237_0_2_7.png
99911_0_2_15.png
102563_0_0_10.png
101517_3_2_7.png
100459_0_0_10.png
103519_2_2_3.png
101509_1_2_12.png
102522_3_1_12.png
102521_3_2_18.png
104027_1_0_17.png
100331_1_0_2.png
102523_1_2_17.png
100331_1_2_10.png
104268_0_2_3.png
102598_3_0_14.png
101164_1_0_16.png
101509_0_2_15.png
99911_3_2_10.png
102563_3_2_14.png
102237_0_1_6.png
102563_3_0_7.png
101569_3_1_15.png
101517_3_1_8.png
100631_1_1_5.png
102521_0_0_2.png
101548_1_2_1.png
102601_1_1_1.png
102237_3_1_10.png
103303_3_2_6.png
100331_3_0_13.png
101550_0_1_13.png
103303_3_0_15.png
99495_0_2_17.png
102521_0_2_18.png
104008_0_1_1.png
9949

In [75]:
trap_dir = "Datasets/backup/Only_traps"
back_dir = "Datasets/Crow_detection/val/background"
save_img_dir = "Datasets/Crow_detection_train/images/val"
save_ann_dir = "Datasets/Crow_detection_train/labels/val"
save_gt_dir = "Datasets/Crow_detection/val/GT_BB"

generate_detection_trap_samples(back_dir, trap_dir, save_img_dir, save_ann_dir, save_gt_dir)

N samples:  77
101520_2_0_7.png
103138_3_2_10.png
102522_0_1_12.png
103303_1_2_11.png
104268_2_0_18.png
104357_0_0_8.png
102237_2_2_16.png
99495_3_2_12.png
100459_0_2_14.png
101559_2_2_1.png
103303_0_0_11.png
101559_2_0_16.png
103638_2_0_15.png
102601_1_0_2.png
101569_1_0_3.png
101125_1_1_9.png
102563_1_0_16.png
102601_2_2_11.png
101343_2_2_15.png
101520_0_2_17.png
104270_0_1_5.png
101559_1_0_11.png
102522_1_1_1.png
102521_0_1_17.png
104268_3_1_15.png
104270_1_1_8.png
100615_2_2_3.png
103439_1_1_2.png
104357_2_2_5.png
102522_2_0_6.png
103849_1_0_11.png
102522_1_2_8.png
100631_1_2_6.png
101546_0_0_8.png
102521_1_1_5.png
103519_0_0_6.png
101123_1_2_16.png
101343_1_0_2.png
99911_0_0_5.png
101679_3_2_16.png
102520_3_1_17.png
101559_1_1_17.png
101569_0_2_2.png
103710_2_0_7.png
101442_0_1_10.png
104269_3_1_12.png
99911_2_2_17.png
102523_1_0_10.png
101125_1_2_1.png
101343_1_2_7.png
103439_3_2_13.png
102563_3_1_4.png
102522_0_2_17.png
101548_3_0_5.png
101679_2_0_6.png
101550_1_2_15.png
101517_

##### General augmentation

In [9]:
from torchvision.transforms.autoaugment import AutoAugmentPolicy

transform_p = 0.8

def apply_general_augmentation(samples_dir, save_dir, p_to_generate = 0.65, auto_augment_policy = AutoAugmentPolicy.SVHN, replace=True):


    transform = transforms.Compose(
        [
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.5),
            transforms.AutoAugment(auto_augment_policy),
            transforms.ColorJitter(
                brightness=0.5, contrast=0.15, saturation=0.3, hue=0.3
            ),
        ]
    )


    samples_list = random.choices(
        os.listdir(samples_dir), k=int(len(os.listdir(samples_dir)) * p_to_generate)
    )

    for sample in samples_list:

        sample_img = Image.open(os.path.join(samples_dir, sample))
        sample_img = transform(sample_img)
        
        sample_name = sample.split(".")[0] + "_aug" + ".jpeg"
        if replace:
            sample_name = sample

        print(sample_name)
        sample_img.save(os.path.join(save_dir, sample_name))

apply_general_augmentation("Datasets/Crow_classify/train/background", "Datasets/Crow_classify/train/background", p_to_generate = 0.2, replace=False)
apply_general_augmentation("Datasets/Crow_classify/train/crow_trap", "Datasets/Crow_classify/train/crow_trap", p_to_generate = 0.2, replace=False)

# apply_general_augmentation("Datasets/Crow_classify/train/background", "Datasets/Crow_classify/train/background", p_to_generate = 0.2, replace=True)
# apply_general_augmentation("Datasets/Crow_classify/train/crow_trap", "Datasets/Crow_classify/train/crow_trap", p_to_generate = 0.2, replace=True)

101343_2_0_aug.jpeg
103710_1_0_aug.jpeg
103138_1_0_aug.jpeg
104357_2_2_aug.jpeg
101520_3_0_aug.jpeg
102523_3_1_aug.jpeg
104008_1_2_aug.jpeg
101164_0_0_aug.jpeg
102237_1_0_aug.jpeg
103710_0_1_aug.jpeg
101559_2_0_aug.jpeg
99911_3_0_aug.jpeg
103138_3_2_aug.jpeg
101679_3_1_aug.jpeg
104268_1_2_aug.jpeg
103439_2_2_aug.jpeg
99495_1_2_aug.jpeg
104269_1_0_aug.jpeg
102237_2_2_aug.jpeg
101509_0_2_aug.jpeg
100615_1_1_aug.jpeg
101550_0_2_aug.jpeg
103303_2_0_aug.jpeg
99911_1_1_aug.jpeg
101343_0_0_aug.jpeg
101548_1_2_aug.jpeg
104270_0_0_aug.jpeg
104268_0_2_aug.jpeg
103303_0_2_aug.jpeg
101509_3_0_aug.jpeg
104269_1_2_aug.jpeg
101442_2_2_aug.jpeg
100980_1_0_aug.jpeg
102522_2_2_aug.jpeg
102521_1_0_aug.jpeg
101679_0_2_aug.jpeg
104268_3_1_aug.jpeg
101442_2_2_aug.jpeg
102520_1_2_aug.jpeg
101517_0_1_aug.jpeg
104270_0_0_aug.jpeg
100631_3_0_aug.jpeg
104272_2_0_aug.jpeg
102520_1_0_aug.jpeg
103439_0_1_aug.jpeg
101125_0_2_aug.jpeg
102521_3_0_aug.jpeg
100459_0_0_aug.jpeg
100631_3_0_aug.jpeg
104357_3_2_aug.jpeg
103

### Split data into train-val

In [10]:
make_split("crow_trap", dataset_dir="Datasets/Crow_classify/train", split_p=.15)
make_split("background", dataset_dir="Datasets/Crow_classify/train", split_p=.15)

Sample list:  406
345 61
Sample list:  453
385 68


### Convert to greyscale

In [12]:
def to_grey(samples_dir, save_dir):

    samples_list = os.listdir(samples_dir)

    for sample in samples_list:

        sample_img = Image.open(os.path.join(samples_dir, sample))
        to_greyscale = transforms.Grayscale()
        sample_img = to_greyscale(sample_img)
        
        sample_img.save(os.path.join(save_dir, sample))

to_grey("Datasets/Crow_classify_grey/train/background", "Datasets/Crow_classify_grey/train/background")
to_grey("Datasets/Crow_classify_grey/train/crow_trap", "Datasets/Crow_classify_grey/train/crow_trap")

to_grey("Datasets/Crow_classify_grey/val/background", "Datasets/Crow_classify_grey/val/background")
to_grey("Datasets/Crow_classify_grey/val/crow_trap", "Datasets/Crow_classify_grey/val/crow_trap")

to_grey("Datasets/Crow_evaluate_classify_grey/val/background", "Datasets/Crow_evaluate_classify_grey/val/background")
to_grey("Datasets/Crow_evaluate_classify_grey/val/crow_trap", "Datasets/Crow_evaluate_classify_grey/val/crow_trap")